In [5]:
import scipy.io as sio
import numpy as np
from os import getcwd
from os.path import join 
import matplotlib.pyplot as plt
import os
import statsmodels.api as sm
from statsmodels.multivariate.pca import PCA
from statsmodels.tsa.ar_model import AutoReg
import pandas as pd 

In [7]:
fname = join(os.getcwd(),'data','data3SS2009.mat')

mat_contents = sio.loadmat(fname)

dataset = mat_contents['dataset']


N, Chno, Nc = dataset.shape
# N: number of samples
# Chno: number of channels
# Nc: number of cases

y = mat_contents['labels'].reshape(Nc)




In [18]:
Ch1 = dataset[:,0,:] # célula de carga: força do shaker
Ch2 = dataset[:,1,:] # acelerômetro: base
Ch3 = dataset[:,2,:] # acelerômetro: 1o andar
Ch4 = dataset[:,3,:] # acelerômetro: 2o andar
Ch5 = dataset[:,4,:] # acelerômetro: 3o andar

Ts = 3.125 * 1e-3 # sampling time
time = (np.linspace(1,N,N) - 1) * Ts


    


In [9]:
ARmodel_2 = AutoReg(Ch2[:,1],30,old_names=False).fit()
ARmodel_3 = AutoReg(Ch3[:,1],30,old_names=False).fit()
ARmodel_4 = AutoReg(Ch4[:,1],30,old_names=False).fit()
ARmodel_5 = AutoReg(Ch5[:,1],30,old_names=False).fit()
vec_2 = ARmodel_2.params
vec_3 = ARmodel_3.params
vec_4 = ARmodel_4.params
vec_5 = ARmodel_5.params
vec_temp = np.concatenate((vec_2,vec_3,vec_4,vec_5))

ARmodel_var = vec_temp

for i in range(1,850):
    ARmodel_2 = AutoReg(Ch2[:,i],30,old_names=False).fit()
    ARmodel_3 = AutoReg(Ch3[:,i],30,old_names=False).fit()
    ARmodel_4 = AutoReg(Ch4[:,i],30,old_names=False).fit()
    ARmodel_5 = AutoReg(Ch5[:,i],30,old_names=False).fit()
    vec_2 = ARmodel_2.params
    vec_3 = ARmodel_3.params
    vec_4 = ARmodel_4.params
    vec_5 = ARmodel_5.params
    vec_temp = np.concatenate((vec_2,vec_3,vec_4,vec_5))
    ARmodel_var = np.vstack([ARmodel_var,vec_temp])

In [10]:
X = ARmodel_var
X.shape

(850, 124)

In [12]:
import numpy as np
from scipy import stats
from scipy.stats.stats import mode
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import (train_test_split, RepeatedKFold,
RandomizedSearchCV)
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)


In [14]:
## SVM Method

from scipy.stats import randint, expon 
from sklearn.svm import SVC

estimator = SVC()
param_dist = {
    'C': expon(scale=1),  # Regularization parameter (C)
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Kernel type
    'degree': randint(1, 10),  # Degree of the polynomial kernel
    'gamma': ['scale', 'auto'] + list(expon(scale=0.1).rvs(20)),  
}

cv = RepeatedKFold(n_splits=5, n_repeats=50, random_state=42)

random_search = RandomizedSearchCV(
    estimator=estimator,
    param_distributions=param_dist,
    n_iter=100,
    scoring='accuracy', 
    cv=cv,
    n_jobs=-1,  
    random_state=42
)



In [15]:

random_search.fit(X, y)  

print("Best Hyperparameters:", random_search.best_params_)
print("Best Accuracy:", random_search.best_score_)

Best Hyperparameters: {'C': 4.088216525459695, 'degree': 9, 'gamma': 0.01681009892720547, 'kernel': 'linear'}
Best Accuracy: 0.9973647058823529


In [16]:
## kNN Method
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import randint, expon 


estimator = KNeighborsClassifier()
param_dist = {
    'n_neighbors': randint(1, 30),  # Number of neighbors
    'weights': ['uniform', 'distance'], 
    'p': [1, 2],  
}

cv = RepeatedKFold(n_splits=5, n_repeats=50, random_state=42)

random_search = RandomizedSearchCV(
    estimator=estimator,
    param_distributions=param_dist,
    n_iter=100,
    scoring='accuracy', 
    cv=cv,
    n_jobs=-1,  
    random_state=42
)

random_search.fit(X, y)  

print("Best Hyperparameters:", random_search.best_params_)
print("Best Accuracy:", random_search.best_score_)

Best Hyperparameters: {'n_neighbors': 4, 'p': 1, 'weights': 'distance'}
Best Accuracy: 0.9743294117647058


In [ ]:
from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform 

estimator = LogisticRegression(multi_class='multinomial', solver='saga', max_iter=1000)
param_dist = {
    'C': uniform(0.1, 10.0),  
}

cv = RepeatedKFold(n_splits=5, n_repeats=50, random_state=42)

random_search = RandomizedSearchCV(
    estimator=estimator,
    param_distributions=param_dist,
    n_iter=100,
    scoring='accuracy',  
    cv=cv,
    n_jobs=-1,  
    random_state=42
)
random_search.fit(X, y)
print("Best Hyperparameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

In [ ]:
# The results found are not the same as those found with the default configurations (cf Take-Home Exercice 3)
# Generally speaking, accuracy is better after searching for the best hyperpartameters.
# Model with LogisticRegression has a calculation time too long

